<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/tidy_up_preprocessing_notebook/notebooks/processed/ct_preprocessing_ubs_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
===================================================
Author: Chiaki Tachikawa
Role: Data Science Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/chiaki-tachikawa
Date: 2025-02-13
Version: 1.1

Description:
    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:
    - Importing necessary libraries and downloading NLTK data.
    - Defining and applying a `preprocessor` function to clean and tokenize text data.
    - Reading and preprocessing various CSV files containing transcript data.
    - Exporting the preprocessed data to new CSV files for further analysis.

===================================================
"""

'\n===================================================\nAuthor: Chiaki Tachikawa\nRole: Data Science Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/chiaki-tachikawa\nDate: 2025-02-13\nVersion: 1.1\n\nDescription:\n    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:\n    - Importing necessary libraries and downloading NLTK data.\n    - Defining and applying a `preprocessor` function to clean and tokenize text data.\n    - Reading and preprocessing various CSV files containing transcript data.\n    - Exporting the preprocessed data to new CSV files for further analysis.\n\n===================================================\n'

# **Library**

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from google.colab import drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# **Function**

preprocessor function : The function modifies the DataFrame data in place, adding two new columns (col1 and col2) with preprocessed text.


Input:
  - name of dataframe
  - name of column which contains the text to clean
  - name of column which is tokenized
  - name of column which is cleaned

In [ ]:
#create function to preprocess data
def preprocessor (data, col, col1,col2):
  #Copy col1umn
  data[col1]=data[col]
  data[col2]=data[col]


  #Adding column1
  #Lower the lettercase
  data[col1] = data[col1].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col1] = data[col1].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #Tokenize the word
  data[col1] = data[col1].apply(nltk.word_tokenize)

  #Remove numbers
  data[col1] = data[col1].apply(lambda x: [word for word in x if not word.isdigit()])

  #remove symbol from comments
  data[col1] = data[col1].apply(lambda x: [word for word in x if x!=""])

  #remove short word
  data[col1] = data[col1].apply(lambda x: [word for word in x if len(word)>2])

  #remove symbols
  data[col1] = data[col1].apply (lambda x: [re.sub(r"[^a-z]", "", word) for word in x])

  #lemmatization
  lemmatizer = WordNetLemmatizer()
  data[col1] = data[col1].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])



  #Adding column2
  #Lower the lettercase
  data[col2] = data[col2].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col2] = data[col2].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #remove symbols
  data[col2] = data[col2].apply (lambda x: [re.sub(r"[.,'?]", "", x)])

  return


find_row: This function searches upwards from the given current_row_num in the DataFrame df to find the first row where the value in column "M" is "A". It returns the index of that row. If no such row is found, it returns 0

In [ ]:
def find_row (df, col, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col][i] == "A":
      break
    else:
      i-=1
  return i

find_row_empty: This function searches upwards from the given current_row_num to find the first row where col1 has the value "A" and col2 is not an empty list. It returns the index of that row.

In [ ]:
def find_row_empty (df, col1, col2, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col1][i] == "A" and df[col2][i] != []:
      break
    else:
      i-=1
  return i

In [ ]:
def create_ques_num_column (data, new_col,marker_col):
  #Create question number column
  data[new_col]=None
  #set global var to count question number
  num = 0
  #if Q was found, num adds 1 otherwise none
  for i in data.index:
    if data.loc[i,marker_col]=="Q":
      data.at[i,new_col]=num
      num +=1
    else:
      continue

In [ ]:
# Function to extract names
def extract_name(full_string):
    return full_string.split(',')[0]

In [ ]:
#check if there is "A" before "Q" from the current location
def find_last_a (df, col, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col][i] == "Q":
      j = i-1
      while j > 0:
        if df[col][j]=="A":
          pass
        else:
          break
        j-=1
      break
    else:
      i-=1
  return i

## **Data**

In [ ]:
#drive.mount('/content/drive')

In [ ]:
#!ls"/content/bank_of_england/data/preprocessed_data/Archived/jpmorgan_qa_section_preprocessed.csv"

In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


JP Morgan QA section

In [ ]:
"""
#Obtaining management discussion / git bash
!git clone https://github.com/sheldonkemper/bank_of_england.git
!git switch Preprocessing
%cd bank_of_england/data/preprocessed_data/archived
%ls
"""

'\n#Obtaining management discussion / git bash\n!git clone https://github.com/sheldonkemper/bank_of_england.git\n!git switch Preprocessing\n%cd bank_of_england/data/preprocessed_data/archived\n%ls\n'

In [ ]:
#Defining qa_data
qa_data = pd.read_csv("/content/drive/MyDrive/bank_of_england/data/cleansed/ubs_qna_section.csv")
qa_data.head()

,speaker,job_title,utterance,call_date,financial_quarter,source_file
0,Unknown,NaN,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
1,Sergio P. Ermotti,NaN,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
2,Chris Hallam,Goldman Sachs,"Very clear. Thanks. Kian Abouhossein, JPMorgan...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
3,Sergio P. Ermotti,NaN,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
4,Sarah Youngwood,NaN,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf


In [ ]:
#ascend df by financial_quarter
quarter_order = ["1Q23", "2Q23","3Q23","4Q23","1Q24","2Q24","3Q24","4Q24"]
qa_data["financial_quarter"] = pd.Categorical(qa_data["financial_quarter"], categories=quarter_order, ordered=True)
qa_data = qa_data.sort_values("financial_quarter", kind="mergesort")

In [ ]:
#preprocessing data
preprocessor(qa_data, "utterance", "question_tokenised_data", "question_cleaned")
preprocessor(qa_data,"utterance","answer_tokenised_data","answer_cleaned")

In [ ]:
#remove operater
#qa_data = qa_data.loc[qa_data["speaker"]!="Operator"]

#remove less than 20 words
qa_data["count"] = qa_data["question_tokenised_data"].apply(lambda x: len(x))
qa_data = qa_data.loc[qa_data["count"]>20]
qa_data.head()

In [ ]:
#reset index
qa_data.reset_index(drop=True, inplace=True)
qa_data.head()

In [ ]:
#Create question number column
create_ques_num_column(qa_data,"question_number_inline","marker")

In [ ]:
qa_data.head()

In [ ]:
#adding new columns
qa_data["analyst"]=None
qa_data["analyst_title"]=None
qa_data["metadata_question"]=None
qa_data["question_num"]=None

#

for i in qa_data.index:
    name_list=[]
    title_list=[]
    question_list=[]
    quest_num_list=[]
    #If the value in the "marker" column at row i is "Q", then set the values of "analyst", "analyst_title", "metadata_question", and "question_num" columns at row i to "x".
    if qa_data.loc[i,"marker"]=="Q":
      qa_data.at[i,"analyst"]="x"
      qa_data.at[i,"analyst_title"]="x"
      qa_data.at[i,"metadata_question"]="x"
      qa_data.at[i,"question_num"]="x"
    elif find_last_a(qa_data,"marker",i) >=0 and qa_data.loc[i-1,"marker"]=="Q":
      name_list.append(qa_data["speaker"][i-1])
      title_list.append(qa_data["job_title"][i-1])
      question_list.append(qa_data["utterance"][i-1])
      quest_num_list.append(str(qa_data["question_number_inline"][i-1]))
    elif find_last_a(qa_data,"marker",i) >=0 and qa_data.loc[i-1,"marker"]=="A":
      qa_data.at[i,"analyst"] = qa_data.at[i-1,"analyst"]
      qa_data.at[i, "analyst_title"] = qa_data.at[i-1,"analyst_title"]
      qa_data.at[i,"metadata_question"]=qa_data.at[i-1, "metadata_question"]
      qa_data.at[i,"question_num"] = qa_data.at[i-1, "question_num"]

    #Initialize lists and populate them with "speaker", "job_title", "utterance", and "question_number_inline" values from rows between last_a and i in qa_data.
    else:
      last_a = find_row(qa_data,"marker", i)+ 1
      for j in range(last_a, i):
        name_list.append(qa_data["speaker"][j])
        title_list.append(qa_data["job_title"][j])
        question_list.append(qa_data["utterance"][j])
        quest_num_list.append(str(qa_data["question_number_inline"][j]))
 # Assign the lists name_list, title_list, question_list, and quest_num_list to the "analyst", "analyst_title", "metadata_question", and "question_num" columns at row i in qa_data, respectively.
    qa_data.at[i,"analyst"]=name_list
    qa_data.at[i,"analyst_title"]=title_list
    qa_data.at[i,"metadata_question"]=question_list
    qa_data.at[i,"question_num"]=quest_num_list

In [ ]:
qa_data.head()

In [ ]:
for i in range(len(qa_data)):
  #if the first row is answer, then return all analyst columns as x, if the row is answer and there is not analyst, then popurate questioner analyst name and other data, otherwise pass the function
  if i ==0 and qa_data["marker"][i]=="A":
    qa_data.at[i,"analyst"] ="x"
    qa_data.at[i,"analyst_title"] ="x"
    qa_data.at[i,"metadata_question"] ="x"
    qa_data.at[i,"question_num"]="x"
  elif qa_data["marker"][i]=="A"and qa_data["analyst"][i]==[]:
    a = find_row_empty(qa_data,"marker","analyst",i)
    qa_data.at[i,"analyst"] = qa_data.loc[a,"analyst"]
    qa_data.at[i,"analyst_title"] = qa_data.loc[a,"analyst_title"]
    qa_data.at[i,"metadata_question"] = qa_data.loc[a,"metadata_question"]
    qa_data.at[i,"question_num"] =qa_data.loc[a,"question_num"]
  else:
    continue


In [ ]:
#rename column
qa_data.rename(columns={"utterance":"metadata"},inplace=True)


In [ ]:
#reorganise column
qa_data=qa_data[["filename","financial_quarter","call_date","speaker","marker","question_num","job_title","metadata", "answer_cleaned","analyst","analyst_title","metadata_question","question_cleaned"]]

In [ ]:
# Apply the function to the DataFrame
qa_data['speaker'] = qa_data['speaker'].apply(extract_name)

In [ ]:
"""
#ascend df by financial_quarter
quarter_order = ["1Q23", "2Q23","3Q23","4Q23","1Q24","2Q24","3Q24","4Q24"]
qa_data["financial_quarter"] = pd.Categorical(qa_data["financial_quarter"], categories=quarter_order, ordered=True)
df_sorted = qa_data.sort_values("financial_quarter", kind="mergesort")
"""

In [ ]:
qa_data.head(20)

JP morgan management discussion

In [ ]:
%ls

In [ ]:
#defining santader dataframe
jpmorgan_body_df=pd.read_csv("jpmorgan_management_discussion.csv")
jpmorgan_body_df.head()

In [ ]:
#preprocess data
preprocessor(jpmorgan_body_df, "chunk_text", "tokenized_data","cleaned_data")

In [ ]:
jpmorgan_body_df.head()

UBS qna section

In [ ]:
%ls

In [ ]:
#define ubs q&a data
ubs_qna_df=pd.read_csv("ubs_qna_section.csv")

In [ ]:
#preprocessing ubs Q&A data
preprocessor(ubs_qna_df, "utterance", "tokenized_data","cleaned_data")

In [ ]:
ubs_qna_df.head()

UBS management discussion

In [ ]:
%ls

In [ ]:
#defining ubs management discussion
ubs_manag_df=pd.read_csv("ubs_management_discussion.csv")
ubs_manag_df.head()

In [ ]:
#preprocessing ubs management discussion
preprocessor(ubs_manag_df,"utterance", "tokenized_data","cleaned_data")
ubs_manag_df.head()

# **Export the output as a csv file**

JP morgan QA section

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path1 = "/content/drive/MyDrive/bank_of_england/data/preprocessed_data/jpmorgan_qna_df_preprocessed_ver7.csv"
qa_data.to_csv(preprocessed_qa_csv_path1, index=False)

JP morgan management discussion

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path2 = "/content/sample_data/jpmorgan_management_df_preprocessed.csv"
jpmorgan_body_df.to_csv(preprocessed_qa_csv_path2, index=False)

UBS QA section

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path3 = "/content/sample_data/ubs_qa_df_preprocessed.csv"
ubs_qna_df.to_csv(preprocessed_qa_csv_path3, index=False)

UBS management discussion

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path4 = "/content/sample_data/ubs_management_df_preprocessed.csv"
ubs_manag_df.to_csv(preprocessed_qa_csv_path4, index=False)